In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.decomposition import TruncatedSVD
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Flatten, Dot, Dense, Concatenate
from tensorflow.keras.optimizers import Adam
from sklearn.impute import SimpleImputer

2025-02-21 23:16:52.608541: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1740179812.624560 3996605 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1740179812.629367 3996605 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
############################# Step 1: Data Preprocessing ############################# 

# Define directories for training and validation data
train_dir = "./altis_power_cap_res/2_dual_cap/train"
validation_dir = "./ecp_power_cap_res/2_dual_cap/validation"
save_model_dir = "./altis_power_cap_res/2_dual_cap/model"
os.makedirs(save_model_dir, exist_ok=True)


# Load CSV files from train directory
train_csv_files = [f for f in os.listdir(train_dir) if f.endswith(".csv")]
validation_csv_files = [f for f in os.listdir(validation_dir) if f.endswith(".csv")]

# Load and merge training data
train_data = []
for file in train_csv_files:
    file_path = os.path.join(train_dir, file)
    df = pd.read_csv(file_path)
    train_data.append(df)
train_df = pd.concat(train_data, ignore_index=True)

# Load validation data per application
validation_data = {}
for file in validation_csv_files:
    file_path = os.path.join(validation_dir, file)
    df = pd.read_csv(file_path)
    validation_data[file] = df

# Drop any rows with missing values
train_df.dropna(inplace=True)
for key in validation_data:
    validation_data[key].dropna(inplace=True)


In [6]:
############################# Step 2: Build MLP Model ############################# 

# Define feature columns and target column
feature_cols = ["CPU Power Cap", "GPU Power Cap", "IPS", "Memory Throughput", "SM Clock", "DRAM Active", "FP Active"]
target_col = "Performance"

# Extract features and target for training
X_train = train_df[feature_cols].values
y_train = train_df[target_col].values.reshape(-1, 1)

# Normalize features and target
scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()
X_train_scaled = scaler_X.fit_transform(X_train)
y_train_scaled = scaler_y.fit_transform(y_train)

# Build the improved MLP model
# Define the MLP model properly
model = Sequential([
    tf.keras.Input(shape=(X_train_scaled.shape[1],)),  # Explicitly define input shape
    Dense(256, activation='selu'),
    BatchNormalization(),
    Dropout(0.3),
    Dense(128, activation='selu'),
    BatchNormalization(),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(1, activation='linear')
])

In [14]:
############################# Step 3: Train MLP Model #############################
# Compile the model with a different optimizer
model.compile(optimizer='nadam', loss='mse', metrics=['mae'])

# Train the model
history = model.fit(X_train_scaled, y_train_scaled, epochs=50, batch_size=32, verbose=3)

# Save the trained model
model.save(os.path.join(save_model_dir, "performance_prediction_model.h5"))

Epoch 1/50
Epoch 2/50
Epoch 3/50
Epoch 4/50
Epoch 5/50
Epoch 6/50
Epoch 7/50
Epoch 8/50
Epoch 9/50
Epoch 10/50
Epoch 11/50
Epoch 12/50
Epoch 13/50
Epoch 14/50
Epoch 15/50
Epoch 16/50
Epoch 17/50
Epoch 18/50
Epoch 19/50
Epoch 20/50
Epoch 21/50
Epoch 22/50
Epoch 23/50
Epoch 24/50
Epoch 25/50
Epoch 26/50
Epoch 27/50
Epoch 28/50
Epoch 29/50
Epoch 30/50
Epoch 31/50
Epoch 32/50
Epoch 33/50
Epoch 34/50
Epoch 35/50
Epoch 36/50
Epoch 37/50
Epoch 38/50
Epoch 39/50
Epoch 40/50
Epoch 41/50
Epoch 42/50
Epoch 43/50
Epoch 44/50
Epoch 45/50
Epoch 46/50
Epoch 47/50
Epoch 48/50
Epoch 49/50
Epoch 50/50


In [41]:
############################# Step 4: Evaluate Model Performance ############################# 

model_path = os.path.join(save_model_dir, "performance_prediction_model.h5")  # Legacy format
model = tf.keras.models.load_model(model_path, custom_objects={"mse": tf.keras.losses.MeanSquaredError()})
feature_cols = ["CPU Power Cap", "GPU Power Cap", "IPS", "Memory Throughput", "SM Clock", "DRAM Active", "FP Active"]
target_col = "Performance"


for file in validation_csv_files:
    file_path = os.path.join(validation_dir, file)
    new_app_name = file.replace("_performance.csv", "")

    # Load validation data
    df_new = pd.read_csv(file_path)
    df_new["Power Pair"] = list(zip(df_new["CPU Power Cap"], df_new["GPU Power Cap"]))

    # Add new application to performance matrix
    performance_matrix[new_app_name] = np.nan

    #############  Predict 20% of power pairs using trained NN --- Construct Sparse Matrix  ############# 
    
    df_sampled = df_new.sample(frac=0.2, random_state=42)  # Select 20% of rows
    sampled_pairs = df_sampled["Power Pair"].unique()
    # print(sampled_pairs)
    
    true_values, nn_predicted_values = [], []
    for power_pair in sampled_pairs:
        # Extract corresponding feature values for the selected power pair
        X_sample = df_new[df_new["Power Pair"] == power_pair][feature_cols].values
        
        # Normalize features
        X_sample_scaled = scaler_X.transform(X_sample)
        
        # Predict performance
        y_pred = model.predict(X_sample_scaled)
        predicted_value = scaler_y.inverse_transform(y_pred)[0][0]
    
        # Fill the performance matrix with NN predictions
        performance_matrix.at[power_pair, new_app_name] = predicted_value
    
        # Store true values if available (for validation)
        if power_pair in df_new.set_index("Power Pair").index:
            true_values.append(df_new.loc[df_new["Power Pair"] == power_pair, "Performance"].values[0])
            nn_predicted_values.append(predicted_value)
            
    # Compute Accuracy Metrics
    nn_mae = mean_absolute_error(true_values, nn_predicted_values)
    nn_rmse = np.sqrt(mean_squared_error(true_values, nn_predicted_values))
    nn_r2 = r2_score(true_values, nn_predicted_values)
    
    # Print evaluation results
    print(f"Neural Network Prediction for {app_name} (Sampled 20% Power Pairs, One-by-One Prediction):")
    print(f"MAE: {nn_mae:.4f}, RMSE: {nn_rmse:.4f}, R²: {nn_r2:.4f}\n")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 299ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
Neural Network Prediction for lammps_performance.csv (Sampled 20% Power Pairs, One-by-One Prediction):
MAE: 0.0326, RMSE: 0.0411, R²: 0.8980



In [11]:
############################# Step 5: Populate Initial Performance Matrix ############################# 

model_path = os.path.join(save_model_dir, "performance_prediction_model.h5")  # Legacy format
model = tf.keras.models.load_model(model_path, custom_objects={"mse": tf.keras.losses.MeanSquaredError()})
feature_cols = ["CPU Power Cap", "GPU Power Cap", "IPS", "Memory Throughput", "SM Clock", "DRAM Active", "FP Active"]
target_col = "Performance"

# Dictionary to store application data
app_data = {}

# Process each CSV file (each application)
for file in train_csv_files:
    file_path = os.path.join(train_dir, file)
    app_name = file.replace("_performance.csv", "")
    
    df = pd.read_csv(file_path)
    required_columns = ["CPU Power Cap", "GPU Power Cap", "Performance"]
    df = df[required_columns]

    # Create a unique power pair column
    df["Power Pair"] = list(zip(df["CPU Power Cap"], df["GPU Power Cap"]))
    
    # Store application data
    app_data[app_name] = df[["Power Pair", "Performance"]].set_index("Power Pair")

# Combine all applications into a single 2D matrix
performance_matrix = pd.DataFrame(index=sorted(set().union(*[df.index for df in app_data.values()])),
                                  columns=sorted(app_data.keys()))


# Populate the performance matrix with training data
for app_name, df in app_data.items():
    performance_matrix[app_name] = df["Performance"]

In [12]:
############################# Step 6: construct sparse matrix and predict missing value ############################# 

nn_results = []
cf_results = []

# Process each new application in the validation folder
for file in validation_csv_files:
    file_path = os.path.join(validation_dir, file)
    new_app_name = file.replace("_performance.csv", "")

    # Load validation data
    df_new = pd.read_csv(file_path)
    df_new["Power Pair"] = list(zip(df_new["CPU Power Cap"], df_new["GPU Power Cap"]))

    # Add new application to performance matrix
    performance_matrix[new_app_name] = np.nan

    #############  Predict 20% of power pairs using trained NN --- Construct Sparse Matrix  ############# 
    
    df_sampled = df_new.sample(frac=0.2, random_state=42)  # Select 20% of rows
    sampled_pairs = df_sampled["Power Pair"].unique()
    # print(sampled_pairs)
    
    true_values, nn_predicted_values = [], []
    for power_pair in sampled_pairs:
        # Extract corresponding feature values for the selected power pair
        X_sample = df_new[df_new["Power Pair"] == power_pair][feature_cols].values
        
        # Normalize features
        X_sample_scaled = scaler_X.transform(X_sample)
        
        # Predict performance
        y_pred = model.predict(X_sample_scaled)
        predicted_value = scaler_y.inverse_transform(y_pred)[0][0]
    
        # Fill the performance matrix with NN predictions
        performance_matrix.at[power_pair, new_app_name] = predicted_value
    
        # Store true values if available (for validation)
        if power_pair in df_new.set_index("Power Pair").index:
            true_values.append(df_new.loc[df_new["Power Pair"] == power_pair, "Performance"].values[0])
            nn_predicted_values.append(predicted_value)
    
    ############## Compute NN Prediction Accuracy (Before CF) #############
    
    nn_mae = mean_absolute_error(true_values, nn_predicted_values)
    nn_rmse = np.sqrt(mean_squared_error(true_values, nn_predicted_values))
    nn_r2 = r2_score(true_values, nn_predicted_values)

    nn_results.append((new_app_name, nn_mae, nn_rmse, nn_r2))

    # print(f"Neural Network Prediction for {new_app_name} (Sampled 20% Power Pairs):")
    # print(f"MAE: {nn_mae:.4f}, RMSE: {nn_rmse:.4f}, R²: {nn_r2:.4f}")

    ##############  Train Neural CF  ##############
    power_pair_map = {pair: i for i, pair in enumerate(performance_matrix.index)}
    app_map = {app: i for i, app in enumerate(performance_matrix.columns)}

    train_data = []
    train_labels = []
    
    for app in performance_matrix.columns:
        for power_pair in performance_matrix.index:
            if not np.isnan(performance_matrix.at[power_pair, app]):  # Only use observed values
                train_data.append([power_pair_map[power_pair], app_map[app]])
                train_labels.append(performance_matrix.at[power_pair, app])

    train_data = np.array(train_data)
    train_labels = np.array(train_labels)

    # Define Neural CF Model
    num_power_pairs = len(power_pair_map)
    num_apps = len(app_map)
    latent_dim = 10  # Embedding size

    input_power_pair = Input(shape=(1,))
    input_app = Input(shape=(1,))

    power_embedding = Embedding(num_power_pairs, latent_dim)(input_power_pair)
    app_embedding = Embedding(num_apps, latent_dim)(input_app)

    power_vec = Flatten()(power_embedding)
    app_vec = Flatten()(app_embedding)

    merged = Concatenate()([power_vec, app_vec])
    dense_1 = Dense(64, activation='relu')(merged)
    dense_2 = Dense(32, activation='relu')(dense_1)
    output = Dense(1, activation='linear')(dense_2)

    ncf_model = Model(inputs=[input_power_pair, input_app], outputs=output)
    ncf_model.compile(optimizer=Adam(learning_rate=0.001), loss='mse', metrics=['mae'])
    ncf_model.fit([train_data[:, 0], train_data[:, 1]], train_labels, epochs=50, batch_size=32, verbose=3)

    ##############  Predict Missing Value via ncf  ##############
    for power_pair in performance_matrix.index:
        if np.isnan(performance_matrix.at[power_pair, new_app_name]):
            power_idx = power_pair_map[power_pair]
            app_idx = app_map[new_app_name]
            pred_value = ncf_model.predict([np.array([power_idx]), np.array([app_idx])])[0][0]
            performance_matrix.at[power_pair, new_app_name] = pred_value

    ############### Compute CF Prediction Accuracy ##############
    true_values = df_new.set_index("Power Pair")["Performance"]
    predicted_values = performance_matrix[new_app_name].reindex(true_values.index)

    cf_mae = mean_absolute_error(true_values, predicted_values)
    cf_rmse = np.sqrt(mean_squared_error(true_values, predicted_values))
    cf_r2 = r2_score(true_values, predicted_values)

    cf_results.append((new_app_name, cf_mae, cf_rmse, cf_r2))

    # print(f"\nNeural CF Prediction Accuracy for {new_app_name}:")
    # print(f"MAE: {cf_mae:.4f}, RMSE: {cf_rmse:.4f}, R²: {cf_r2:.4f}")

    # Print True vs. CF Predicted values
    # print(f"\nTrue vs. Predicted Performance for {new_app_name}:")
    # print(pd.DataFrame({"True": true_values, "Predicted": predicted_values}))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 293ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
Epoch 1/50
Epoch 2/50
Epoch 3/50
Epoch 4/50
Epoch 5/50
Epoch 6/50
Epoch 7/50
Epoch 8/50
Epoch 9/50
Epoch 10/50
Epoch 11/50
Epoch 12/50
Epoch 13/50
Epoch 14/50
Epoch 15/50
Epoch 16/50
Epoch 17/50
Epoch 18/50
Epoch 19/50
Epoch 20/50
Epoch 21

In [13]:
# Print stored accuracy metrics after all iterations
print("\n=== Neural Network Prediction Accuracy ===")
for app, mae, rmse, r2 in nn_results:
    print(f"{app}: MAE={mae:.4f}, RMSE={rmse:.4f}, R²={r2:.4f}")

print("\n=== Neural Collaborative Filtering Prediction Accuracy ===")
for app, mae, rmse, r2 in cf_results:
    print(f"{app}: MAE={mae:.4f}, RMSE={rmse:.4f}, R²={r2:.4f}")


=== Neural Network Prediction Accuracy ===
lammps: MAE=0.0541, RMSE=0.0648, R²=0.7459
sw4lite: MAE=0.0874, RMSE=0.1065, R²=0.4824
Laghos: MAE=0.0986, RMSE=0.1247, R²=-4.9145
miniGAN: MAE=0.0566, RMSE=0.0715, R²=0.4395
CRADL: MAE=0.1670, RMSE=0.1821, R²=-1.2521

=== Neural Collaborative Filtering Prediction Accuracy ===
lammps: MAE=0.0428, RMSE=0.0505, R²=0.8305
sw4lite: MAE=0.0910, RMSE=0.1041, R²=0.4728
Laghos: MAE=0.0929, RMSE=0.1103, R²=-3.7772
miniGAN: MAE=0.0509, RMSE=0.0676, R²=0.3004
CRADL: MAE=0.1761, RMSE=0.1895, R²=-1.5822
